# <center> NCAA 2017 Basketball Bracket Creator </center>

This Notebook extracts a bracket from a Kaggle Submission File. It shows what new brackets/predictions look like as the game moves forward, and full rounds have been played out.

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn import metrics

In [33]:
tourney_slots = pd.read_csv('2017_Data/TourneySlots.csv')
teams = pd.read_csv('2017_Data/Teams.csv')
t_seeds = pd.read_csv('2017_Data/TourneySeeds.csv')

In [34]:
submission = pd.read_csv('Submission_2017.csv')

In [35]:
probframe = submission.Id.str.extract('([0-9]+)_([0-9]+)_([0-9]+)').rename_axis({0:'Season',1:'Team_1_id',2:'Team_2_id'},axis=1)
probframe['Pred'] = submission.Pred

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [36]:
teams_dict = teams.set_index(['Team_Id']).to_dict()
probframe['Team_1_Name'] = probframe.apply(lambda x:(teams_dict['Team_Name'][int(x.Team_1_id)]), axis =1)
probframe['Team_2_Name'] = probframe.apply(lambda x:(teams_dict['Team_Name'][int(x.Team_2_id)]), axis =1)
probframe.head()
probframe= probframe[['Season', 'Team_1_id', 'Team_2_id', 'Team_1_Name',
       'Team_2_Name', 'Pred']]
probframe.head()

,Season,Team_1_id,Team_2_id,Team_1_Name,Team_2_Name,Pred
0,2017,1181,1437,Duke,Villanova,0.228692
1,2017,1124,1437,Baylor,Villanova,0.124324
2,2017,1196,1437,Florida,Villanova,0.113139
3,2017,1437,1438,Villanova,Virginia,0.820983
4,2017,1374,1437,SMU,Villanova,0.243500


In [37]:
t_seeds_2017 = t_seeds[t_seeds.Season==2017]
tourney_slots_2017 = tourney_slots[tourney_slots.Season==2017]

In [38]:
def get_round_winners(game_slots, team_seeds, season='2017', known_winners = []):
    seed_list = []
    season_list = []
    team_list = []
    team_1_list = []
    team_2_list = []
    prob_list = []
    prob_label_list = []
    result = []
    
    def create_row(game):
        t1 = team_seeds[(team_seeds.Seed == game.Strongseed)].Team.iloc[0]
        t2 = team_seeds[(team_seeds.Seed == game.Weakseed)].Team.iloc[0]
        if(t1>t2):
            t1,t2 = t2, t1
        t1_name = teams[teams.Team_Id == t1].Team_Name.iloc[0]
        t2_name = teams[teams.Team_Id == t2].Team_Name.iloc[0]
        team_1_list.append(t1_name + ' ('+ str(t1)+')')
        team_2_list.append(t2_name + ' ('+ str(t2)+')')
        prob = probframe[(probframe.Team_1_id==str(t1)) & (probframe.Team_2_id==str(t2)) & (probframe.Season==season)].Pred.iloc[0]
        seed_list.append(game.Slot)
        season_list.append(season)
        prob_list.append(prob)
        prob_label_list.append(season+'_'+str(t1)+'_'+str(t2))
        if(t1 in known_winners):
            winner = t1
            result.append(1)
        elif (t2 in known_winners):
            winner  = t2
            result.append(0)
        else:
            winner = t1 if prob>0.5 else t2
            result.append(int(round(prob)))
        team_list.append(winner)
        return winner
    game_slots.apply(create_row, axis=1)
    return pd.DataFrame({'Season':season_list, 'Seed':seed_list, 'Team':team_list, 'Team 1':team_1_list, 'Team 2':team_2_list,'Probability':prob_list, 'Submission_Label':prob_label_list, 'Result':result},
                       columns= ['Season','Seed','Team','Team 1','Team 2', 'Probability', 'Submission_Label','Result'])

teams_dict2 = teams.set_index(['Team_Name']).to_dict()
print('First Four Predictions')
display(get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^[XWYZ]')],
                          t_seeds_2017[t_seeds_2017.Seed.str.match(r'.*[ab]$')],
                         known_winners= [teams_dict2['Team_Id'][x] for x in ['USC', 'Mt St Mary\'s', 'Kansas St','UC Davis'] ]))


First Four Predictions


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result
0,2017,W11,1425,Providence (1344),USC (1425),0.528060,2017_1344_1425,0
1,2017,W16,1291,Mt St Mary's (1291),New Orleans (1309),0.580722,2017_1291_1309,1
2,2017,Y16,1413,NC Central (1300),UC Davis (1413),0.475509,2017_1300_1413,0
3,2017,Z11,1243,Kansas St (1243),Wake Forest (1448),0.566606,2017_1243_1448,1


In [39]:
#Set winners of First Four
t_seeds_2017 = t_seeds_2017.append(get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^[XWYZ]')], 
                       t_seeds_2017[t_seeds_2017.Seed.str.match(r'.*[ab]$')],
                      known_winners= [teams_dict2['Team_Id'][x] for x in ['USC', 'Mt St Mary\'s', 'Kansas St','UC Davis'] ])).drop(['Team 1', 'Team 2', 'Probability', 'Submission_Label'], axis=1)

### March Madness Bracket

In [40]:
def display_winner_name(frame, title):
    print(title)
    display(frame.merge(teams[['Team_Id','Team_Name']], how ='left', left_on =['Team'], right_on=['Team_Id']).drop(['Team_Id'],axis=1)
            .rename(index=str, columns={'Team_Name':'Winner'}))

r1 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R1')], t_seeds_2017)
r2 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R2')], r1)
r3 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R3')], r2)
r4 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R4')], r3)
r5 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R5')], r4)
r6 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R6')], r5)


display_winner_name(r1,'Round 1 Picks')
display_winner_name(r2,'Round 2 Picks')
display_winner_name(r3,'Sweet 16 Picks')
display_winner_name(r4,'Elite Eight Picks')
display_winner_name(r5,'Final Four Picks')
display_winner_name(r6,'Champion')


Round 1 Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R1W1,1437,Mt St Mary's (1291),Villanova (1437),0.003977,2017_1291_1437,0,Villanova
1,2017,R1W2,1181,Duke (1181),Troy (1407),0.983745,2017_1181_1407,1,Duke
2,2017,R1W3,1124,Baylor (1124),New Mexico St (1308),0.809837,2017_1124_1308,1,Baylor
3,2017,R1W4,1196,ETSU (1190),Florida (1196),0.196970,2017_1190_1196,0,Florida
4,2017,R1W5,1438,UNC Wilmington (1423),Virginia (1438),0.224719,2017_1423_1438,0,Virginia
5,2017,R1W6,1374,SMU (1374),USC (1425),0.866787,2017_1374_1425,1,SMU
6,2017,R1W7,1376,Marquette (1266),South Carolina (1376),0.432273,2017_1266_1376,0,South Carolina
7,2017,R1W8,1458,Virginia Tech (1439),Wisconsin (1458),0.200306,2017_1439_1458,0,Wisconsin
8,2017,R1X1,1211,Gonzaga (1211),S Dakota St (1355),0.976668,2017_1211_1355,1,Gonzaga
9,2017,R1X2,1112,Arizona (1112),North Dakota (1315),0.981149,2017_1112_1315,1,Arizona


Round 2 Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R2W1,1437,Villanova (1437),Wisconsin (1458),0.860398,2017_1437_1458,1,Villanova
1,2017,R2W2,1181,Duke (1181),South Carolina (1376),0.851040,2017_1181_1376,1,Duke
2,2017,R2W3,1374,Baylor (1124),SMU (1374),0.305695,2017_1124_1374,0,SMU
3,2017,R2W4,1438,Florida (1196),Virginia (1438),0.369440,2017_1196_1438,0,Virginia
4,2017,R2X1,1211,Gonzaga (1211),Northwestern (1321),0.908335,2017_1211_1321,1,Gonzaga
5,2017,R2X2,1112,Arizona (1112),St Mary's CA (1388),0.699144,2017_1112_1388,1,Arizona
6,2017,R2X3,1199,Florida St (1199),Maryland (1268),0.532777,2017_1199_1268,1,Florida St
7,2017,R2X4,1452,Notre Dame (1323),West Virginia (1452),0.393377,2017_1323_1452,0,West Virginia
8,2017,R2Y1,1242,Kansas (1242),Miami FL (1274),0.872026,2017_1242_1274,1,Kansas
9,2017,R2Y2,1257,Louisville (1257),Michigan (1276),0.719160,2017_1257_1276,1,Louisville


Sweet 16 Picks


...

,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R4W1,1437,SMU (1374),Villanova (1437),0.243500,2017_1374_1437,0,Villanova
1,2017,R4X1,1211,Arizona (1112),Gonzaga (1211),0.464454,2017_1112_1211,0,Gonzaga
2,2017,R4Y1,1242,Kansas (1242),Oregon (1332),0.562405,2017_1242_1332,1,Kansas
3,2017,R4Z1,1314,Kentucky (1246),North Carolina (1314),0.476243,2017_1246_1314,0,North Carolina


Final Four Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R5WX,1437,Gonzaga (1211),Villanova (1437),0.317918,2017_1211_1437,0,Villanova
1,2017,R5YZ,1314,Kansas (1242),North Carolina (1314),0.490503,2017_1242_1314,0,North Carolina


Champion


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R6CH,1437,North Carolina (1314),Villanova (1437),0.361161,2017_1314_1437,0,Villanova


### March Madness Brackets After Round 1

The model preformed pretty well in the first round with 29/32 predicted successfully. It even got several upsets right like MTSU beating Minnesota, Wichita beating Dayton and Rhode Island beating Creighton. The upsets it didn't see comming were Xavier beating Maryland, USC beating SMU and Miami beating Michigan St.(However the Michigan, Miami game was predicted to be a very close game).


Predictions after first round upsets are below:

In [41]:
r1 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R1')], t_seeds_2017,
                       known_winners=[teams_dict2['Team_Id'][x] for x in ['Michigan St', 'USC', 'Xavier']])
r2 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R2')], r1)
r3 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R3')], r2)
r4 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R4')], r3)
r5 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R5')], r4)
r6 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R6')], r5)


#display_winner_name(r1,'Round 1 Picks')
display_winner_name(r2,'Round 2 Picks')
display_winner_name(r3,'Sweet 16 Picks')
display_winner_name(r4,'Elite Eight Picks')
display_winner_name(r5,'Final Four Picks')
display_winner_name(r6,'Champion')


Round 2 Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R2W1,1437,Villanova (1437),Wisconsin (1458),0.860398,2017_1437_1458,1,Villanova
1,2017,R2W2,1181,Duke (1181),South Carolina (1376),0.851040,2017_1181_1376,1,Duke
2,2017,R2W3,1124,Baylor (1124),USC (1425),0.740032,2017_1124_1425,1,Baylor
3,2017,R2W4,1438,Florida (1196),Virginia (1438),0.369440,2017_1196_1438,0,Virginia
4,2017,R2X1,1211,Gonzaga (1211),Northwestern (1321),0.908335,2017_1211_1321,1,Gonzaga
5,2017,R2X2,1112,Arizona (1112),St Mary's CA (1388),0.699144,2017_1112_1388,1,Arizona
6,2017,R2X3,1199,Florida St (1199),Xavier (1462),0.655973,2017_1199_1462,1,Florida St
7,2017,R2X4,1452,Notre Dame (1323),West Virginia (1452),0.393377,2017_1323_1452,0,West Virginia
8,2017,R2Y1,1242,Kansas (1242),Michigan St (1277),0.901246,2017_1242_1277,1,Kansas
9,2017,R2Y2,1257,Louisville (1257),Michigan (1276),0.719160,2017_1257_1276,1,Louisville


Sweet 16 Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R3W1,1437,Villanova (1437),Virginia (1438),0.820983,2017_1437_1438,1,Villanova
1,2017,R3W2,1181,Baylor (1124),Duke (1181),0.323286,2017_1124_1181,0,Duke
2,2017,R3X1,1211,Gonzaga (1211),West Virginia (1452),0.679688,2017_1211_1452,1,Gonzaga
3,2017,R3X2,1112,Arizona (1112),Florida St (1199),0.758074,2017_1112_1199,1,Arizona
4,2017,R3Y1,1242,Kansas (1242),Purdue (1345),0.773828,2017_1242_1345,1,Kansas
5,2017,R3Y2,1332,Louisville (1257),Oregon (1332),0.383715,2017_1257_1332,0,Oregon
6,2017,R3Z1,1314,Butler (1139),North Carolina (1314),0.175750,2017_1139_1314,0,North Carolina
7,2017,R3Z2,1246,Kentucky (1246),UCLA (1417),0.718083,2017_1246_1417,1,Kentucky


Elite Eight Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R4W1,1437,Duke (1181),Villanova (1437),0.228692,2017_1181_1437,0,Villanova
1,2017,R4X1,1211,Arizona (1112),Gonzaga (1211),0.464454,2017_1112_1211,0,Gonzaga
2,2017,R4Y1,1242,Kansas (1242),Oregon (1332),0.562405,2017_1242_1332,1,Kansas
3,2017,R4Z1,1314,Kentucky (1246),North Carolina (1314),0.476243,2017_1246_1314,0,North Carolina


Final Four Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R5WX,1437,Gonzaga (1211),Villanova (1437),0.317918,2017_1211_1437,0,Villanova
1,2017,R5YZ,1314,Kansas (1242),North Carolina (1314),0.490503,2017_1242_1314,0,North Carolina


Champion


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R6CH,1437,North Carolina (1314),Villanova (1437),0.361161,2017_1314_1437,0,Villanova


### March Madness Bracket After Round 2

The second round came in with a bevy of upsets for our model especially in the the East Region. Florida beat Virginia, South Carolina beat Duke and Wisconsin beat Villanova, dethroning our predicted Champion. With Villinova out of the running it appears the predicted teams in the Final will be Gonzaga and North Carolina, with North Carolina having the slight upper hand. 
<br><br>
South Carolina and Duke was interesting, because while Duke had a strong favorability of winning South Carolina had Home Field advantage. Home advantage was not something we used in our model because tournaments are usually played on neutral grounds. However as one of many companies that stood against North Carolina's bathroom laws against Transgender individuals (back in 2016), the NCAA had moved the game from it's intended location, North Carolina to South Sarolina. While there is obviously no way to know due to the volume of other factors that during games, given what is known it is certainly possible Home Field advantage played a role.<br>
http://freakonomics.com/2011/12/18/football-freakonomics-how-advantageous-is-home-field-advantage-and-why/
<br><br>
Another really interesting team that won was Xavier. The 11th seed team not only beat Maryland, but came with an overwhelming victory against 3rd seed Florida State in Round 2 (91-66). One thing that makes Xavier distinct is it's 1-3-1 defense. It typically is not a popular defence, due to its spacial concepts being difficult to teach. However, done right it can throw teams that are not prepared for a loop. Five Thirty Eight does a good job explaining why Xavier's 1-3-1 defense made it special back in 2016. The team continues to still use this zoning pattern<br>
https://fivethirtyeight.com/features/five-weird-teams-to-watch-in-the-ncaa-tournament/<br>
http://www.si.com/college-basketball/2017/03/18/xavier-musketeers-fsu-seminoles-march-madness

Predictions after second round upsets are below:

In [42]:
r1 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R1')], t_seeds_2017,
                        known_winners=[teams_dict2['Team_Id'][x] for x in ['Michigan St', 'USC', 'Xavier']])
r2 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R2')], r1,
                        known_winners=[teams_dict2['Team_Id'][x] for x in ['South Carolina', 'Michigan',
                                                                           'Wisconsin', 'Xavier', 'Florida']])
r3 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R3')], r2)
r4 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R4')], r3)
r5 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R5')], r4)
r6 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R6')], r5)


#display_winner_name(r1,'Round 1 Picks')
#display_winner_name(r2,'Round 2 Picks')
display_winner_name(r3,'Sweet 16 Picks')
display_winner_name(r4,'Elite Eight Picks')
display_winner_name(r5,'Final Four Picks')
display_winner_name(r6,'Champion')


Sweet 16 Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R3W1,1458,Florida (1196),Wisconsin (1458),0.440521,2017_1196_1458,0,Wisconsin
1,2017,R3W2,1124,Baylor (1124),South Carolina (1376),0.731502,2017_1124_1376,1,Baylor
2,2017,R3X1,1211,Gonzaga (1211),West Virginia (1452),0.679688,2017_1211_1452,1,Gonzaga
3,2017,R3X2,1112,Arizona (1112),Xavier (1462),0.856688,2017_1112_1462,1,Arizona
4,2017,R3Y1,1242,Kansas (1242),Purdue (1345),0.773828,2017_1242_1345,1,Kansas
5,2017,R3Y2,1332,Michigan (1276),Oregon (1332),0.195382,2017_1276_1332,0,Oregon
6,2017,R3Z1,1314,Butler (1139),North Carolina (1314),0.175750,2017_1139_1314,0,North Carolina
7,2017,R3Z2,1246,Kentucky (1246),UCLA (1417),0.718083,2017_1246_1417,1,Kentucky


Elite Eight Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R4W1,1458,Baylor (1124),Wisconsin (1458),0.466223,2017_1124_1458,0,Wisconsin
1,2017,R4X1,1211,Arizona (1112),Gonzaga (1211),0.464454,2017_1112_1211,0,Gonzaga
2,2017,R4Y1,1242,Kansas (1242),Oregon (1332),0.562405,2017_1242_1332,1,Kansas
3,2017,R4Z1,1314,Kentucky (1246),North Carolina (1314),0.476243,2017_1246_1314,0,North Carolina


Final Four Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R5WX,1211,Gonzaga (1211),Wisconsin (1458),0.742167,2017_1211_1458,1,Gonzaga
1,2017,R5YZ,1314,Kansas (1242),North Carolina (1314),0.490503,2017_1242_1314,0,North Carolina


Champion


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R6CH,1314,Gonzaga (1211),North Carolina (1314),0.453607,2017_1211_1314,0,North Carolina


### March Madness Bracket After Sweet 16

Big upsets: South Carolina beats Baylor, Xavier beats Arizona. 
Reasonable Upsets: Florida Beats Wisconsin

In [43]:
r1 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R1')], t_seeds_2017,
                        known_winners=[teams_dict2['Team_Id'][x] for x in ['Michigan St', 'USC', 'Xavier']])
r2 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R2')], r1,
                        known_winners=[teams_dict2['Team_Id'][x] for x in ['South Carolina', 'Michigan',
                                                                           'Wisconsin', 'Xavier', 'Florida']])
r3 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R3')], r2,
                        known_winners=[teams_dict2['Team_Id'][x] for x in ['Florida', 'South Carolina', 'Xavier', 'Florida']])
r4 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R4')], r3)
r5 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R5')], r4)
r6 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R6')], r5)


#display_winner_name(r1,'Round 1 Picks')
#display_winner_name(r2,'Round 2 Picks')
#display_winner_name(r3,'Sweet 16 Picks')
display_winner_name(r4,'Elite Eight Picks')
display_winner_name(r5,'Final Four Picks')
display_winner_name(r6,'Champion')


Elite Eight Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R4W1,1196,Florida (1196),South Carolina (1376),0.710646,2017_1196_1376,1,Florida
1,2017,R4X1,1211,Gonzaga (1211),Xavier (1462),0.872839,2017_1211_1462,1,Gonzaga
2,2017,R4Y1,1242,Kansas (1242),Oregon (1332),0.562405,2017_1242_1332,1,Kansas
3,2017,R4Z1,1314,Kentucky (1246),North Carolina (1314),0.476243,2017_1246_1314,0,North Carolina


Final Four Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R5WX,1211,Florida (1196),Gonzaga (1211),0.215128,2017_1196_1211,0,Gonzaga
1,2017,R5YZ,1314,Kansas (1242),North Carolina (1314),0.490503,2017_1242_1314,0,North Carolina


Champion


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R6CH,1314,Gonzaga (1211),North Carolina (1314),0.453607,2017_1211_1314,0,North Carolina


### March Madness Bracket After Elite Eight

USC scored an upset over Florida. Oregon, which had about a 44% probability, beats with Kansas

In [44]:
r1 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R1')], t_seeds_2017,
                        known_winners=[teams_dict2['Team_Id'][x] for x in ['Michigan St', 'USC', 'Xavier']])
r2 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R2')], r1,
                        known_winners=[teams_dict2['Team_Id'][x] for x in ['South Carolina', 'Michigan',
                                                                           'Wisconsin', 'Xavier', 'Florida']])
r3 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R3')], r2,
                        known_winners=[teams_dict2['Team_Id'][x] for x in ['Florida', 'South Carolina', 'Xavier']])
r4 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R4')], r3,
                        known_winners=[teams_dict2['Team_Id'][x] for x in ['South Carolina', 'Oregon']])
r5 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R5')], r4)
r6 = get_round_winners(tourney_slots_2017[tourney_slots_2017.Slot.str.match(r'^R6')], r5)


#display_winner_name(r1,'Round 1 Picks')
#display_winner_name(r2,'Round 2 Picks')
#display_winner_name(r3,'Sweet 16 Picks')
#display_winner_name(r4,'Elite Eight Picks')
display_winner_name(r5,'Final Four Picks')
display_winner_name(r6,'Champion')


Final Four Picks


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R5WX,1211,Gonzaga (1211),South Carolina (1376),0.899458,2017_1211_1376,1,Gonzaga
1,2017,R5YZ,1314,North Carolina (1314),Oregon (1332),0.571413,2017_1314_1332,1,North Carolina


Champion


,Season,Seed,Team,Team 1,Team 2,Probability,Submission_Label,Result,Winner
0,2017,R6CH,1314,Gonzaga (1211),North Carolina (1314),0.453607,2017_1211_1314,0,North Carolina


### March Madness Conclusion

The model successfully picked the winners for both the Final Four and the Championship

### Log Loss of Predictions

Computing the log loss of the model after the results of the Championship, we can figure out how well the model did by comparing it to the Kaggle Leaderboard. The log loss successfully passes the .5 Benchmark and the Seed Bench Mark. This particular Submission File, would score around 90-100 out of 442 competing teams. Not bad.

https://www.kaggle.com/c/march-machine-learning-mania-2017/leaderboard

In [45]:
metrics.log_loss(r1.Result.append(r2.Result).append(r3.Result).append(r4.Result).append(r5.Result).append(r6.Result), r1.Probability.append(r2.Probability).append(r3.Probability).append(r4.Probability).append(r5.Probability).append(r6.Probability).values)

0.49282608802503425